In [ ]:
# To mount
from google.colab import drive
drive.mount('/content/drive')

# # To initialize the work environment
# %cd /content/drive/My Drive/
# !git clone https://github.com/allnightlight/ConditionalWassersteinAutoencoderPoweredBySinkhornDistance/casestudies -b casestudies

# # To update the work environment
# %cd /content/drive/My Drive/ConditionalWassersteinAutoencoderPoweredBySinkhornDistance
# !git pull

%cd /content/drive/My Drive/ConditionalWassersteinAutoencoderPoweredBySinkhornDistance/casestudies

In [ ]:
import os
import sys
sys.path.append("../framework/")
sys.path.append("../sl/")
sys.path.append("../wae/")

import torch
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

from conc_environment_factory import ConcEnvironmentFactory
from conc_build_parameter import ConcBuildParameter
from conc_build_parameter_factory import ConcBuildParameterFactory
from conc_agent_factory import ConcAgentFactory
from wae_trainer_factory import WaeTrainerFactory


from builder import Builder
from store import Store
from mylogger import MyLogger

from loader import Loader

In [ ]:
dbPath = "training_log.sqlite"

In [ ]:
target_casestudy = "cs01a"

In [ ]:
def evaluate_err(buildParameter, agent):
    environment = environmentFactory.create(buildParameter)
    trainer = trainerFactory.create(buildParameter, agent, environment)

    err_latent = []
    err_observable = []
    for dataBatchEnv in environment.generateBatchDataIterator():
        dataBatchAg = agent(dataBatchEnv)
        _err_latent, _ = trainer.measure_distance(dataBatchAg._Xi, dataBatchAg._XiHat)
        _err_observable = torch.mean(torch.abs(dataBatchAg._XHat - dataBatchEnv._X))

        err_latent.append(_err_latent.data.numpy())
        err_observable.append(_err_observable.data.numpy())
    err_latent = np.mean(err_latent)
    err_observable = np.mean(err_observable)
    
    return err_observable, err_latent

## S400: Load trained agents to analyze them

### SS410: initialize a loader of trained agents

In [ ]:
agentFactory = ConcAgentFactory()
environmentFactory = ConcEnvironmentFactory()
trainerFactory =  WaeTrainerFactory()

store = Store(dbPath)

buildParameterFactory = ConcBuildParameterFactory()
loader = Loader(agentFactory=agentFactory
                , environmentFactory=environmentFactory
                , buildParameterFactory=buildParameterFactory
                , store = store)

### SS420: evaluate trained agents:

evaluation error is here:
* the representitive errors of observable variables
* and the discrepancy between latent referenced distribution and the one projected by trained encoder

In [ ]:
tbl = {
    "representative_error": []
    , "latent_distribution_discrepancy": []
      }
for agent, buildParameter, epoch in loader.load(target_casestudy + "%"):
    
    for key in buildParameter.__dict__:
        if not key in tbl:
            tbl[key] = []
        tbl[key].append(buildParameter.__dict__[key])
    
    err_observable, err_latent = evaluate_err(buildParameter, agent)
    
    tbl["representative_error"].append(err_observable)
    tbl["latent_distribution_discrepancy"].append(err_latent)
    
tbl = pd.DataFrame(tbl)
tbl.to_csv(target_casestudy +  "_error.csv")